In [1]:
import sys
sys.path.append('../../..')

In [11]:
import torch

from omegaconf import OmegaConf
from transformers import (AutoTokenizer, 
                          AutoModelForSeq2SeqLM, 
                          DataCollatorForSeq2Seq, 
                          Seq2SeqTrainingArguments, 
                          Seq2SeqTrainer)
from tqdm import tqdm

from src.utils import seed_everything
from src.data_prepocessing import load_ds, tokenize_ds
from src.evaluation import Evaluator

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Experiment setup

In [4]:
config = OmegaConf.load("t5_large_1_config.yaml")
print(OmegaConf.to_yaml(config))

model_name: ai-forever/ruT5-large
training_args:
  eval_strategy: epoch
  learning_rate: 0.0002
  batch_size: 16
  gradient_accumulation_steps: 16
  weight_decay: 0.01
  save_total_limit: 3
  num_train_epochs: 50
  predict_with_generate: true
  fp16: false
  push_to_hub: false
  logging_steps: 10
  overwrite_output_dir: true
lora_args:
  r: 4
  lora_alpha: 8
  lora_dropout: 0.1
  target_modules:
  - q
  - v
  bias: none



In [5]:
seed_everything(42)

# Model and data loading

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(config.model_name).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
def preprocess_function(sample):
    inputs = [f"{context[-1]}</s>{phrase}</s>"
    for context, phrase in zip(sample["history"], sample["phrase"])]

    model_inputs = tokenizer(inputs, text_target=sample["rewrite"], max_length=128, truncation=True)
    return model_inputs

In [8]:
ds = load_ds("2rca_checked_version.json")
tokenized_ds = tokenize_ds(ds, preprocess_function)

Map: 100%|██████████| 551/551 [00:00<00:00, 4073.73 examples/s]


# Model training

In [9]:
from peft import LoraConfig, get_peft_model
from peft.utils.peft_types import TaskType


lora_config = LoraConfig(
    r=config.lora_args.r,
    lora_alpha=config.lora_args.lora_alpha,
    lora_dropout=config.lora_args.lora_dropout,
    target_modules=config.lora_args.target_modules,
    bias=config.lora_args.bias,
    task_type=TaskType.SEQ_2_SEQ_LM
)

peft_model = get_peft_model(model, lora_config)

In [10]:
peft_model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 738,847,744 || trainable%: 0.1597


In [ ]:
training_args = Seq2SeqTrainingArguments(
    overwrite_output_dir=config.training_args.overwrite_output_dir,
    eval_strategy=config.training_args.eval_strategy,
    learning_rate=config.training_args.learning_rate,
    per_device_train_batch_size=config.training_args.batch_size,
    per_device_eval_batch_size=config.training_args.batch_size,
    gradient_accumulation_steps=config.training_args.gradient_accumulation_steps,
    weight_decay=config.training_args.weight_decay,
    save_total_limit=config.training_args.save_total_limit,
    num_train_epochs=config.training_args.num_train_epochs,
    predict_with_generate=config.training_args.predict_with_generate,
    fp16=config.training_args.fp16,
    push_to_hub=config.training_args.push_to_hub,
    logging_steps=config.training_args.logging_steps,
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=config.model_name
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["val"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

<ipython-input-22-c5cb18e81d1c>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pvlshkunov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,4.409600,2.866761
2,3.355000,1.617104
3,2.190500,1.248271
4,1.717100,1.096961
5,1.377500,1.001582
6,1.386100,0.931949
7,1.277200,0.876736
8,1.200400,0.832450
9,1.125100,0.798288
10,1.005400,0.768079


Epoch,Training Loss,Validation Loss
1,4.409600,2.866761
2,3.355000,1.617104
3,2.190500,1.248271
4,1.717100,1.096961
5,1.377500,1.001582
6,1.386100,0.931949
7,1.277200,0.876736
8,1.200400,0.832450
9,1.125100,0.798288
10,1.005400,0.768079


KeyboardInterrupt: 

In [ ]:
model = peft_model.merge_and_unload()

# Evaluation

In [ ]:
def infer_ds(ds, model, **kwargs):
    raw_test_results = []
    model = model.to(DEVICE)

    for enc in tqdm(ds['test']['input_ids']):
        input_ids=torch.tensor([enc]).to(DEVICE)
        out = model.generate(inputs=input_ids,
                             eos_token_id=tokenizer.eos_token_id,
                             max_length=config.inference_args.max_length,
                             num_beams=config.inference_args.num_beams,
                             early_stopping=config.inference_args.early_stopping)
        out = tokenizer.decode(out[0][1:],
                               skip_special_tokens=config.inference_args.skip_special_tokens,
                               clean_up_tokenization_spaces=config.inference_args.clean_up_tokenization_spaces)
        raw_test_results.append(out)

    return raw_test_results

In [ ]:
evaluator = Evaluator(dataset=tokenized_ds, 
                      model=model, 
                      tokenizer=tokenizer, 
                      infer_func=infer_ds)

evaluator.evaluate()

,bleu_score,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rf_score_1,rf_score_2,rf_score_3,rf_score_4
type,,,,,,,,,,
2rca,73.335229,0.740367,0.643178,0.565795,0.49352,0.739559,0.285681,0.21532,0.188449,0.176948
